In [1]:
import torch

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error
from datetime import timedelta

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Path to directories
path = '/content/drive/MyDrive/TFM - Neural ODEs/Neural ODE/df.csv'

# Read data
date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df = pd.read_csv(path, sep=',', index_col='datetime', parse_dates=['datetime'], date_parser=date_parser)
df.head()

Mounted at /content/drive


<ipython-input-2-f9765ec2bc53>:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')


,demand,wind,price,day_of_week
datetime,,,,
2018-01-01 00:00:00,22718.1,11487.7,6.74,7
2018-01-01 01:00:00,21510.8,10123.2,4.74,7
2018-01-01 02:00:00,19865.8,8763.5,3.66,7
2018-01-01 03:00:00,19248.3,5661.2,2.30,7
2018-01-01 04:00:00,18632.1,5689.0,2.30,7


In [3]:
# From Friday to Friday
df = df.loc['2019-08-01 00:00':'2019-10-31 23:00'] # 3 months Test

# train_spot = train_df.values
df

,demand,wind,price,day_of_week
datetime,,,,
2019-08-01 00:00:00,27650.6,6482.9,50.15,3
2019-08-01 01:00:00,25939.8,5957.8,45.05,3
2019-08-01 02:00:00,24911.6,5353.0,42.00,3
2019-08-01 03:00:00,24345.8,4766.9,42.15,3
2019-08-01 04:00:00,23826.0,4402.3,42.50,3
...,...,...,...,...
2019-10-31 19:00:00,31139.3,5766.3,60.00,3
2019-10-31 20:00:00,30879.4,6003.4,56.26,3
2019-10-31 21:00:00,29581.3,6786.1,53.55,3


FRIDAY

In [4]:
# Dictionary to map day of week to its string representation
day_of_week_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday',
                    4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

# Dictionary to store the global MAE for each day
global_mae_dict = {}

# Function to calculate MAE
def calculate_mae(actual, forecast):
    return mean_absolute_error(actual, forecast)

def calculate_and_plot(df, day_of_week):
    # Filter the dataframe for the day_of_week
    day_df = df[df.index.dayofweek == day_of_week].copy()

    # Add a 'Date' and 'Hour' column to the dataframe
    day_df['Date'] = day_df.index.date
    day_df['Hour'] = day_df.index.hour

    # Reshape the dataframe to have one row per date and 24 columns for each hour
    day_df = day_df.pivot(index='Date', columns='Hour', values='price')

    mae_list = []

    # Iterate over each day
    for i in range(1, len(day_df)):  # starting from 1 to be able to get the previous observation

        # Get the actual prices for the day
        actual = day_df.iloc[i].values

        # Get the date of the preceding day
        preceding_day = pd.to_datetime(day_df.index[i]) - timedelta(days=1)

        # Get the forecasted prices (the prices of the preceding day)
        forecast = df.loc[str(preceding_day.date())]['price'].values

        # Skip this iteration if there are any NaN values in actual or forecast
        if np.isnan(actual).any() or np.isnan(forecast).any():
            continue

        # Calculate MAE for this day
        mae = calculate_mae(actual, forecast)
        mae_list.append(mae)

        # Plot the actual values and predictions for this day
        plt.figure(figsize=(10, 5))
        plt.plot(range(24), actual, 'bo-', label='Actual')
        plt.plot(range(24), forecast, 'ro-', label='Prediction')

        plt.title(f"Actual vs Prediction for {day_of_week_dict[day_of_week]} {day_df.index[i]} \n MAE: {mae:.2f}")
        plt.xlabel('Hour of the Day')
        plt.xticks(range(24))
        plt.ylabel('Electricity Price')
        plt.legend()
        plt.grid(True)
        plt.show()

    # Calculate global MAE for the day
    global_mae = np.mean(mae_list)
    print(f"Global Mean Absolute Error (MAE) for {day_of_week_dict[day_of_week]}s: {global_mae:.2f}")
    return global_mae

# Call the function for each day of the week
for i in range(7):
    global_mae_dict[i] = calculate_and_plot(df, i)

# Display the global MAE for each day
for day, mae in global_mae_dict.items():
    print(f"Global Mean Absolute Error (MAE) for {day_of_week_dict[day]}s: {mae:.2f}")

Output hidden; open in https://colab.research.google.com to view.